In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

#Properties of the fluid
K = 0.001                           #Pipe roughness [mm]
n = 1.31*10**-6                     #Viscosity [m2/s]
g = 9.81                            #Gravity [m2/s]
E = 784532117.25*100                #Pipe's elasticity [Pa]
Kf = 1.96*10**9                     #Fluid's elasticity [Pa]
ρ = 999.4                           #Fluid's density [kg/m3]
μ = 0.3                             #Pipe's freedom

#Initial conditions
elevd = 10
elevup = 30
Q = 1.988
D = 0.77
e = 0.01
l = 1000
hfl = 10                   #Percentage of local losses [%]

#Calculation conditions
nodes = 11
tclose = 0
hzero = 120
tmax = 50

Din = D - 2 * e 
A = (np.pi * Din ** 2) * 0.25
v = Q/A

def fcw(n, D, v, K):
    midf = []
    Re = v * D / n
    f = (1/(2*np.log10(K/(3.72*D*1000))))**2
    midf.append(f)
    for i in range(1, 10):
        midf.append((1/(2*np.log10((K/(3.72*D*1000)) + (2.51/(Re*np.sqrt(midf[i - 1]))))))**2)
    f = midf[-1]
    return f

f = fcw(n, Din, v, K)

hf = f*l*v**2/(Din*2*g)
hl = hfl/100*hf
dh = hf + hl

a = np.sqrt(Kf/ρ)/(np.sqrt(1 + (Kf * D * (1 - μ**2))/(E * e)))
dx = l/(nodes - 1)
dt = dx/a
delel = (elevup - elevd)/(nodes - 1)
ca = (g * np.pi * D**2)/(4 * a)
step = np.arange(0, tmax + dt, dt)

x = np.zeros(nodes)
h = np.zeros(nodes)
hlow = np.zeros(nodes)
hhigh = np.zeros(nodes)
pipez = np.zeros(nodes)
head = np.zeros(nodes)

for i in range(nodes):
    x[i] = i * dx
    h[i] = hzero - dh * i / nodes
    hlow[i] = h[i]
    hhigh[i] = h[i]
    pipez[i] = elevup - delel * i
    head[i] = h[i] - pipez[i]

Qic = np.ones(nodes) * Q
Vic = np.ones(nodes) * v
Hic = h

def chars(nodes, Q, D, n, K, H, ca, dt, hzero, V0, t, tclose):
    def fcw(n, D, v, K):
        midf = []
        Re = v * D / n
        f = (1/(2*np.log10(K/(3.72*D*1000))))**2
        midf.append(f)
        for i in range(1, 10):
            midf.append((1/(2*np.log10((K/(3.72*D*1000)) + (2.51/(Re*np.sqrt(midf[i - 1]))))))**2)
        f = midf[-1]
        return f

    cm_ = np.zeros(nodes)
    cn_ = np.zeros(nodes)
    Hnew = np.zeros(nodes)
    Qnew = np.zeros(nodes)
    Vnew = np.zeros(nodes)
    for i in range(1, nodes):
        if Q[i - 1] != 0:
            f = fcw(n, D, v, K)
        else:
            f = 0
        cfa = f * dt / (np.pi * D ** 3/2)
        cm = Q[i - 1] + ca * H[i - 1] - cfa * Q[i - 1] * abs(Q[i - 1])
        cm_[i] = cm
    for i in range(0, nodes - 1):
        if Q[i + 1] != 0:
            f = fcw(n, D, v, K)
        else:
            f = 0
        cfb = f * dt / (np.pi * D ** 3/2)
        cn = Q[i + 1] - ca * H[i + 1] - cfb * Q[i + 1] * abs(Q[i + 1])
        cn_[i] = cn
    Hnew = 0.5 * (cm_ - cn_) / ca
    Hnew[0] = hzero    
    Qnew = cn_ + ca * Hnew
    Vnew = 4 * Qnew / (np.pi * D ** 2)
    Hnew = 0.5 * (cm_ - cn_) / ca
    Hnew[0] = hzero
    if t > tclose:
        Vnew[-1] = 0
    else:
        Vnew[-1] = V0 - t / tclose * V0
    Qnew[-1] = Vnew[-1] * np.pi * D ** 2 / 4
    Hnew[-1] = (cm_[-1] - Qnew[-1]) / ca
    return Hnew, Qnew, Vnew

H = []
Q = []
V = []

H.append(Hic)
Q.append(Qic)
V.append(Vic)

H.append(chars(nodes, Qic, Din, n, K, Hic, ca, dt, hzero, v, dt, tclose)[0])
Q.append(chars(nodes, Qic, Din, n, K, Hic, ca, dt, hzero, v, dt, tclose)[1])
V.append(chars(nodes, Qic, Din, n, K, Hic, ca, dt, hzero, v, dt, tclose)[2])

for i in range(1, len(step)):
    H.append(chars(nodes, Q[i], Din, n, K, H[i], ca, dt, hzero, v, step[i], tclose)[0])
    Q.append(chars(nodes, Q[i], Din, n, K, H[i], ca, dt, hzero, v, step[i], tclose)[1])
    V.append(chars(nodes, Q[i], Din, n, K, H[i], ca, dt, hzero, v, step[i], tclose)[2])

Head = pd.DataFrame(H)
Discharge = pd.DataFrame(Q)
Velocity = pd.DataFrame(V)

Head

,0,1,2,3,4,5,6,7,8,9,10
0,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,108.973482,107.595167,106.216852
1,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,108.973482,107.595167,473.759052
2,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,108.973482,474.483293,473.829219
3,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,475.207287,474.553211,475.207529
4,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,475.931041,475.276957,475.931513,475.277198
...,...,...,...,...,...,...,...,...,...,...,...
420,120.0,120.786285,121.571628,122.358726,123.143000,123.930783,124.713862,125.502199,126.283957,127.072720,337.153736
421,120.0,120.786221,121.573251,122.358534,123.146245,123.930462,124.718725,125.501751,126.290435,127.072144,-82.558512
422,120.0,120.786965,121.573123,122.360765,123.145990,123.934179,124.718342,125.506950,126.289925,-83.126429,-82.582064
423,120.0,120.786901,121.574476,122.360575,123.148695,123.933863,124.722397,125.506507,-83.694824,-83.149868,-83.694344


In [78]:
def plot_hammer(i = 0):
    plt.plot(Head[i], 'g', zorder = 10)
    plt.hlines(0, -10, len(step), 'black', zorder = 9)
    plt.vlines(0, -abs(Head[10].min()), Head[10].max(), 'black', zorder = 8)
    plt.legend(['Transient Head'])
    plt.title('Node {}'.format(i))
    plt.xlabel('Time [sec]')
    plt.ylabel('Head [m]');

interact(plot_hammer, i = (0, (nodes - 1), 1));

interactive(children=(IntSlider(value=0, description='i', max=10), Output()), _dom_classes=('widget-interact',…

In [80]:
def plot_hammer_line(i = 0):
    plt.plot(Head.loc[i], 'g', zorder = 10)
    plt.hlines(0, 0, nodes, 'black', zorder = 9)
    plt.vlines(0, -abs(Head[10].min()), Head[10].max(), 'black', zorder = 8)
    plt.legend(['Transient Head'])
    plt.title('Time {:.3f} sec'.format(i * dt))
    plt.xlabel('Nodes [-]')
    plt.ylabel('Head [m]');
    
interact(plot_hammer_line, i = (0, len(step), 1));

interactive(children=(IntSlider(value=0, description='i', max=424), Output()), _dom_classes=('widget-interact'…